# 02 Inverse Reinforcement Learning

## Intro


### Nomenclature
1. $S$: set of states.
1. $A$: set of actions.
1. $R$: set of rewards (per states).
1. $\pi$: optimal policy.
1. $T^\pi$: policy transition matrix of size $n×n$ containing state transition probabilities for always choosing $\pi(s)$ in each respective state.
 


### Assumptions
1. Finite, small state spaces.
1. We know the expert's policy $\pi$. 
1. $\pi$ is optimal and stationary deterministic.


### Linear problem
Get the rewards $R$ for the states under an optimal policy $\pi$.

$$
\forall \mathbf{T}^i \in \mathbf{T}^{\neg\pi}: (\mathbf{T}^\pi-\mathbf{T}^i)(\mathbf{I} - \gamma\mathbf{T}^\pi)^{-1} \mathbf{R} \succeq 0
$$

## Create the optimum policy Q

In [1]:
import time

import numpy as np
from scipy import sparse
from scipy.sparse import linalg
from scipy.optimize import linprog

from board import Board
from players.minimax import Minimax
from players.qlearner import QLearner
from utils import test_players

In [2]:
agent =  QLearner(1)

In [3]:
# Train the agent
TRAIN_SECONDS = 180

games_played = 0
time_start = time.time()
time_stop = time.time() + TRAIN_SECONDS
while time.time() < time_stop:
    agent._autotrain_1_game(
        lr=0.1, 
        f_df=lambda x: 1-(x/10),
        board=Board()
    )
    games_played += 1

print("\t {} games played\n".format(games_played))

	 152546 games played



In [4]:
# Test the agent, should tie (-1) against an optimum player (i.e. minimax)
minimax = Minimax(1)
N_GAMES = 100

print("Testing agent VS minimax ({} games)".format(N_GAMES))
print("\tAs player 1")
print(test_players(agent, minimax, N_GAMES))
print("\tAs player 2")
print(test_players(agent, minimax, N_GAMES))

Testing agent VS minimax (100 games)
	As player 1
{1: 0, 2: 0, -1: 100}
	As player 2
{1: 0, 2: 0, -1: 100}


## Linear problem

Original formula:
$$
\forall \mathbf{T}^i \in \mathbf{T}^{\neg\pi}: (\mathbf{T}^\pi-\mathbf{T}^i)(\mathbf{I} - \gamma\mathbf{T}^\pi)^{-1} \mathbf{R} \succeq 0
$$

Formula adapted to solver:
$$
\forall \mathbf{T}^i \in \mathbf{T}^{\neg\pi}: (\mathbf{T}^i-\mathbf{T}^\pi)(\mathbf{I} - \gamma\mathbf{T}^\pi)^{-1} \mathbf{R} \preceq 0
$$

*Notes:*
* T_factor1 $= (\mathbf{T}^i-\mathbf{T}^\pi)$ 

In [5]:
Q = agent.Q

T_pi = {  # Transition probabilities under optimal policy (always 1)
    #'val': [], # Transition probability (1)
    'row': [], # Origin state
    'col': []  # Destination state
}

T_factor1 = {  # Precalculated inecuation (T^i - T^pi)
    # T^i is 1, T^pi is -1, ultra-sparse matrix
    'val': [],
    'row': [],
    'col': []
}

# Index state->position in matrix
index_states = {state:state_i for state_i,state in enumerate(Q.keys())}

def get_position(state):  # TODO, will be a method in a class
    '''Get the position of state in the index.
    If not in the index, append at the end.
    '''
    retval = index_states.get(state)  # Destination state
    if not retval:  # If state not in Q (final state), add to index
        retval = len(index_states)
        index_states[state] = retval
    return retval
    

try:
    ineq_i = 0  # inequation index
    for state,state_i in list(index_states.items()):  # Make it a list, dict will change on the run
        # Populate T_pi, take the best action and add it.
        actions = list(Q[state].items())   # [(action, score)]
        best_action = max(actions, key=lambda x: x[1])
        row,col = best_action[0] # Move in the board
        board = Board(board=state)
        board.move(row,col,1)  # Defensive programming, better execute action than creating m
        #T_pi['val'].append(1)  # Transition is deterministic 1
        T_pi['row'].append(state_i) # Orgin state
        state_i_dest_opt = get_position(board.get_state())
        T_pi['col'].append(state_i_dest_opt)

        actions.remove(best_action)
        # Calculate T_i lines for that state, non-optimal moves
        for (row,col),score in actions:
            board = Board(board=state)
            board.move(row,col,1)
            # Non-optimal move
            T_factor1['row'].append(ineq_i)
            T_factor1['col'].append(get_position(board.get_state()))
            T_factor1['val'].append(1) # 1-0
            # Optimal move
            T_factor1['row'].append(ineq_i)
            T_factor1['col'].append(state_i_dest_opt)
            T_factor1['val'].append(-1) # 0-1    
            ineq_i += 1 # Next inequation 
except Exception as e:
    print(state)
    print(actions)
    print(e)

In [6]:
n_states = len(index_states)

M_factor1 = sparse.csr_matrix(
    (T_factor1['val'], (T_factor1['row'], T_factor1['col'])),
    shape=(max(T_factor1['row'])+1, n_states)
)

In [7]:
DISCOUNT_FACTOR = 0.1

M_factor2 = linalg.inv(
    sparse.identity(n_states) - 
    (DISCOUNT_FACTOR * sparse.csr_matrix( 
        (np.ones(len(T_pi['row'])), (T_pi['row'], T_pi['col'])),
        shape=(n_states, n_states)
    ))
)

/home/samuelmunoz/beeva/github/ml-ai_games/venv/lib/python3.5/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:295: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/home/samuelmunoz/beeva/github/ml-ai_games/venv/lib/python3.5/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:202: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  'is in the CSC matrix format', SparseEfficiencyWarning)


In [8]:
M_ineq = M_factor1.dot(M_factor2)

In [9]:
print("Number of states: "+str(n_states))
print("Shape inequations matrix: ")
print(M_ineq.shape)

Number of states: 7413
Shape inequations matrix: 
(11623, 7413)


In [10]:
result = linprog(
    c = np.ones(n_states),  # Function coefficients
    A_ub = M_ineq.todense(),          # Constraint coefficients, this<=_
    b_ub = np.zeros(M_ineq.shape[0])  # Constraint values _<=this 
)
print(result)

     fun: -0.0
 message: 'Optimization terminated successfully.'
     nit: 0
   slack: array([0., 0., 0., ..., 0., 0., 0.])
  status: 0
 success: True
       x: array([0., 0., 0., ..., 0., 0., 0.])


### A degenerate solution

The problem is I get a degenerate solution.

Rewards equal to zero is a solution to the inequations. To filter this solution I will try to change the function to optimize.

$$
\text{maximize} : \sum_{s\in\mathcal{S}} (Q^\pi(s, \pi(s))-\text{max}_{a\in\mathcal{A}\setminus \pi(s)}Q^\pi(s, a))
$$

Maximize the difference between the first and second best solutions.

Expressed as a minimization problem:
$$
\text{minimize} : \sum_{s\in\mathcal{S}} (Q^\pi(s, \pi(s))+\text{max}_{a\in\mathcal{A}\setminus \pi(s)}Q^\pi(s, a))
$$


In [53]:
coeficients = np.zeros(n_states)  # How should I model final states? No transitions.

for state,state_i in list(index_states.items()):  # Make it a list, dict will change on the run
    # Populate T_pi, take the best action and add it.
    actions = list(Q[state].items())   # [(action, score)]
    if actions:
        best_action = _,score1 = max(actions, key=lambda x: x[1])
        actions.remove(best_action)
        if actions:
            _,score2 = max(actions, key=lambda x: x[1])
        else:
            score2 = 0
        coeficients[state_i] = score1 + score2

Upper bound for Rewards (coeficients)
$$
\forall s \in \mathcal{S}: |\hat{\mathcal{R}}(s)| \leq R_{\text{max}}
$$

In [89]:
M_ineq_coeffs = np.identity(n_states)
R_max = 1 # No reward higher than 1

In [90]:
Z = M_ineq.todense()
Z = np.vstack([Z,M_ineq_coeffs])

In [93]:
print("Number of states: "+str(n_states))
print("Shape inequations matrix: ")
print(Z.shape)

Number of states: 7413
Shape inequations matrix: 
(19036, 7413)


In [91]:
result = linprog(
    c = coeficients,  # Function coefficients, minimize this function
    A_ub = Z,          # Constraint coefficients, this<=_
    b_ub = np.array(
        [0]*M_ineq.shape[0] +
        [R_max]*n_states
    )  # Constraint values _<=this 
)
print(result)

     fun: -312.19582386834895
 message: 'Optimization terminated successfully.'
     nit: 481
   slack: array([0., 0., 0., ..., 1., 1., 1.])
  status: 0
 success: True
       x: array([0., 0., 0., ..., 0., 0., 0.])


In [106]:
list(result.x!=0).count(True)  # States better than 0

327

In [109]:
rev = {i:board for board,i in index_states.items()} # Reverse index state(int) -> board(tuple)
R = {rev[i]:v for i,v in enumerate(result.x) if v!=0}  # State(board) -> Reward

*Note*: if there are same number of 1s and 2s, the last move was made by player 2, it is a desirable state for them. 
Otherwise, more 1s than 2s, player 1 was the last to move, they try to reach this state.

If player 1 reaches this state, they will win.

In [125]:
print(R.get( (0,1,2,1,1,2,2,0,1), 0))  # 1 Will win for sure!

1.0


In [126]:
print(R.get( (0, 1, 2, 0, 1, 2, 2, 1, 1), 0))  # 1 is winner!
print(R.get( (1, 0, 0, 1, 2, 1, 1, 2, 2), 0))  # 1 is winner!

1.0
1.0


Symmetric states should have the same reward

In [123]:
print(R.get((2, 0, 0, 0, 0, 0, 1, 0, 1),0))
print(R.get((2, 0, 1, 0, 0, 0, 0, 0, 1),0))

1.0
1.0


But other non-optimal shouldn't.

In [124]:
print(R.get((2, 0, 0, 0, 1, 0, 0, 0, 1),0))

0


Serialize and save the rewards.

In [134]:
import pickle

with open("rewards.pickle", "+wb") as fd:
    pickle.dump( R, fd )

In [135]:
# Test

with open("rewards.pickle", "rb") as fd:
    Q = pickle.load( fd )

## Resources
* [Tutorial](https://thinkingwires.com/posts/2018-02-13-irl-tutorial-1.html)
* [Optimization library](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html)
* [Simplex doc](https://docs.scipy.org/doc/scipy/reference/optimize.linprog-simplex.html)
* [Linear programming implementation](https://github.com/yrlu/irl-imitation/blob/master/lp_irl.py)